In [1]:
import pandas as pd
import numpy as np
from flask import Flask, request
from pymongo import MongoClient
from bson.objectid import ObjectId
from bson.json_util import dumps
import ast
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity as distance
import seaborn as sns

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/TryAgain/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
sia = SentimentIntensityAnalyzer()

In [3]:
client = MongoClient("mongodb://localhost/Conversations")
db = client.get_database()

# Recomendador

In [57]:
# Diccionario con todas las conversaciones


groups = list(db['Conversations'].distinct('Group'))
dicGroup = {}
for group in groups:
    convGroup = list(db['Conversations'].find({'Group':group}, {'Message'}))
    for dialogue in convGroup[0]['Message']:
        dia = {group:dialogue['message']}
        dicGroup.update(dia)

In [72]:
username = 'Homer Simpson'
users = list(db['Conversations'].distinct('Characters'))
print(type(users))
for user in users:
    print(user)
    if username == user['username']:
        print('ok')

<class 'list'>
{'username': 'Bart Simpson', '_id': ObjectId('5e52c725b6fcdc030722c3c0')}
{'username': 'Homer Simpson', '_id': ObjectId('5e52c725b6fcdc030722c3c3')}
ok
{'username': 'Lisa Simpson', '_id': ObjectId('5e52c725b6fcdc030722c3be')}
{'username': 'Marge Simpson', '_id': ObjectId('5e52c725b6fcdc030722c3c4')}
{'username': 'Martin Prince', '_id': ObjectId('5e52c725b6fcdc030722c3bf')}
{'username': 'Milhouse Van Houten', '_id': ObjectId('5e52c725b6fcdc030722c3c2')}
{'username': 'Moe Szyslak', '_id': ObjectId('5e52c725b6fcdc030722c3c5')}
{'username': 'Nelson Muntz', '_id': ObjectId('5e52c725b6fcdc030722c3c1')}


In [84]:
def addMessage(chatname, username, message):
    users = list(db['Conversations'].distinct('Characters'))
    if db['Conversations'].count_documents({'Group':chatname}, limit = 1) != 0:
        for user in users:
            if username == user['username']:
                i = list(db['Conversations'].find({'Group':chatname}, {'_id':1}))[0]['_id']
                query = list(db['Conversations'].update({"_id":i},{"$push":{"Message": {'username': username, 'message': message}}}))
                return 'Great. Included'
            else:
                ValueError("User not in the group")
    else:
        ValueError("Dialogue not added")